
<h1 id="Gait-Monitoring-for-Parkinson's-Disease-Patients">Gait Monitoring for Parkinson's Disease Patients<a class="anchor-link" href="#Gait-Monitoring-for-Parkinson's-Disease-Patients">¶</a></h1><p><strong>Mentor</strong>: Arthur Tay</p>
<h2 id="Project-Description">Project Description<a class="anchor-link" href="#Project-Description">¶</a></h2><p>The objective of this project is to monitor the gait patterns for people with Parkinson Disease. We will analyse the gait for PD patients during freezing and non-freezing (normal) events. Various parameters (e.g. cadence, step lengths, stride lengths, etc) related to gait will be derived from motion sensors (accelerometers, gyroscopes, magnetometers) using public datasets of PD patients. We will then investigate which of these parameters is most suitable for classification for freezing of gait in PD patients.</p>
<h2 id="Final-Product">Final Product<a class="anchor-link" href="#Final-Product">¶</a></h2><p>To make use of signal processing algorithms in extracting gait parameters from motion sensors and identifying most suitable parameters for classification of freezing of gait in PD patients.</p>
<h2 id="Methodology">Methodology<a class="anchor-link" href="#Methodology">¶</a></h2><p>Students will first need to review and understands how to extract gait parameters from motion sensors. There are different algorithms in the literature and they will need to investigate which is most suitable. They will then make use of a PD patient public dataset to test out their methods. If time permits, they will learn basic machine learning tools for classification.</p>
<h2 id="GaitDB-Dataset"><a href="https://www.physionet.org/content/gaitdb/1.0.0/">GaitDB Dataset</a><a class="anchor-link" href="#GaitDB-Dataset">¶</a></h2><h3 id="Data-Description">Data Description<a class="anchor-link" href="#Data-Description">¶</a></h3><p>Walking stride interval time series included are from 15 subjects: 5 healthy young adults (23 - 29 years old), 5 healthy old adults (71 - 77 years old), and 5 older adults (60 - 77 years old) with Parkinson's disease. The file name indicates old (o), young (y) or Parkinson's disease (pd). For the old and young subjects, the age (in years) is also included in the filename.</p>
<p>Subjects walked continuously on level ground around an obstacle-free path. The stride interval was measured using ultra-thin, force sensitive resistors placed inside the shoe. The analog force signal was sampled at 300 Hz with a 12 bit A/D converter, using an ambulatory, ankle-worn microcomputer that also recorded the data. Subsequently, the time between foot-strikes was automatically computed. The method for determining the stride interval is a modification of a previously validated method that has been shown to agree with force-platform measures, a “gold” standard.</p>
<p>Data were collected from the healthy subjects as they walked in a roughly circular path for 15 minutes, and from the subjects with Parkinson’s disease as they walked for 6 minutes up and down a long hallway.</p>
<h3 id="Set-Up-and-Imports">Set-Up and Imports<a class="anchor-link" href="#Set-Up-and-Imports">¶</a></h3>


In [ ]:

%load_ext autoreload
%autoreload 2



In [ ]:

from forest import * # basically lazy imports everything we need to care about
import np # main library for numeric calculations
# import pandas as pd - main library for data analysis
# import matplotlib.pyplot as plt - main library for data plotting
from glob import glob # check files
from math import *
# import scipy as sp - STEM software
# from scipy import signal

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter

%matplotlib inline
# to place plotting inside the diagram




<h3 id="Get-Data">Get Data<a class="anchor-link" href="#Get-Data">¶</a></h3>


In [ ]:

gaitdb = []
gaitDBnames = []
for i in sorted(glob("gaitdb/*.txt")):
    gaitDBnames.append(i.split("\")[-1])
    gaitdb.append(pd.read_csv(i, sep="	", names=["time", "stride_time"]))

gaitdb[0]




<h3 id="Plotting-Initial-Graphs">Plotting Initial Graphs<a class="anchor-link" href="#Plotting-Initial-Graphs">¶</a></h3>


In [ ]:

for i in range(len(gaitdb)):
    plt.figure(figsize=(12, 6))
    plt.plot(gaitdb[i]["time"], gaitdb[i]["stride_time"])
    plt.title("Data from "+gaitDBnames[i])
    plt.ylabel("(Postulated) Stride Time")
    plt.xlabel("Time")
    plt.show()




<h3 id="Logistic-Regression-Functions">Logistic Regression Functions<a class="anchor-link" href="#Logistic-Regression-Functions">¶</a></h3>


In [ ]:

from scripts.logReg import *




<h3 id="Variance-based-Analysis">Variance-based Analysis<a class="anchor-link" href="#Variance-based-Analysis">¶</a></h3>


In [ ]:

variances = []
for i in range(15):
    var = gaitdb[i].var()["stride_time"]
    print(gaitDBnames[i][:-5]+":", var)
    variances.append(var)



In [ ]:

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_ylabel("Variance of Stride Time")
ax.set_title("Bar Graph of Variance of Stride Time per patient")
ax.bar([i[:-4] for i in gaitDBnames], variances)
plt.show()




<h4 id="Logistic-Regression-Data-Preparation">Logistic Regression Data Preparation<a class="anchor-link" href="#Logistic-Regression-Data-Preparation">¶</a></h4>


In [ ]:

X = np.array([[1]*15, variances])
Y = np.array([0]*5 + [1]*5 + [0]*5)[np.newaxis].T
theta = np.array([0, 1])
m = 15




<h4 id="Logistic-Regression-Brute-Force">Logistic Regression Brute Force<a class="anchor-link" href="#Logistic-Regression-Brute-Force">¶</a></h4>


In [ ]:

logReg(5000000) # tried twice




<h4 id="Testing-Validity">Testing Validity<a class="anchor-link" href="#Testing-Validity">¶</a></h4>


In [ ]:

theta0, theta1 = tuple(theta)
for i in range(15):
    print(gaitDBnames[i][:-5]+":", sigmoid(theta0 + theta1*variances[i]))

h = sigmoid(theta @ X)
np.round(h)




<h3 id="Final-Plots">Final Plots<a class="anchor-link" href="#Final-Plots">¶</a></h3>


In [ ]:

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_ylabel("Probability of FoG")
ax.set_title("Bar Graph of FoG per patient")
ax.bar([i[:-4] for i in gaitDBnames], h)
plt.show()



In [ ]:

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xlabel("Variance")
ax.set_ylabel("Probability of FoG")
ax.set_title("Plot of Probability against given Variance")
ax.plot(variances, h, "r.")
plt.show()




<h3 id="Precision,-Recall,-Accuracy-and-F1-Scores">Precision, Recall, Accuracy and F1 Scores<a class="anchor-link" href="#Precision,-Recall,-Accuracy-and-F1-Scores">¶</a></h3><p>precision = tp/(tp+fp)
recall = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/15
f1 = 2(recall)(precision)/(recall+precision)</p>


In [ ]:

Y.T[0] == np.round(h)



In [ ]:

accuracy = sum(Y.T[0] == np.round(h))/m
accuracy



In [ ]:

preds = np.round(h)
actuals = Y.T[0]
tp = fp = tn = fn = 0
for i in range(m):
    actual = actuals[i]
    pred = preds[i]
    if actual == 1 and pred == 1: tp += 1
    elif actual == 0 and pred == 1: fp += 1
    elif actual == 0 and pred == 0: tn += 1
    else: fn += 1

print(tp, fp, tn, fn)



In [ ]:

precision = tp/(tp+fp)
precision



In [ ]:

recall = tp/(tp+fn)
recall



In [ ]:

specificity = tn/(tn+fp)
specificity



In [ ]:

f1 = 2*(recall)*(precision)/(recall+precision)
round(f1, 1)




<table>
<thead><tr>
<th>Precision</th>
<th>Recall</th>
<th>Specificity</th>
<th>Accuracy</th>
<th>F1 Score</th>
</tr>
</thead>
<tbody>
<tr>
<td>0.80</td>
<td>0.80</td>
<td>0.90</td>
<td>0.867</td>
<td>0.80</td>
</tr>
</tbody>
</table>



<h2 id="Daphnet-Dataset"><a href="https://archive.ics.uci.edu/ml/datasets/Daphnet+Freezing+of+Gait#:~:text=Data%20Set%20Information%3A,on%20generating%20many%20freeze%20events.">Daphnet Dataset</a><a class="anchor-link" href="#Daphnet-Dataset">¶</a></h2><p>(suggested by Prof Tay)</p>
<h3 id="Data-Description">Data Description<a class="anchor-link" href="#Data-Description">¶</a></h3><p>The Daphnet Freezing of Gait Dataset Freezing of Gait in users with Parkinson disease (hereafter Daphnet Freezing of Gait Dataset) is a dataset devised to benchmark automatic methods to recognize gait freeze from wearable acceleration sensors placed on legs and hip.</p>
<p>The dataset was recorded in the lab with emphasis on generating many freeze events. Users performed there kinds of tasks: straight line walking, walking with numerous turns, and finally a more realistic activity of daily living (ADL) task, where users went into different rooms while fetching coffee, opening doors, etc.</p>
<p>This dataset is the result of a collaboration between the Laboratory for Gait and Neurodynamics, Tel Aviv Sourasky Medical Center, Israel and the Wearable Computing Laboratory, ETH Zurich, Switzerland. Recordings were run at the Tel Aviv Sourasky Medical Center in 2008. The study was approved by the local Human Subjects Review Committee, and was performed in accordance with the ethical standards of the Declaration of Helsinki.</p>
<p>This dataset was collected as part of the EU FP6 project Daphnet, grant number 018474-2.
Additional effort to publish this dataset was supported in part by the EU FP7 project CuPiD, grant number 288516.</p>
<h3 id="Data-Attributes">Data Attributes<a class="anchor-link" href="#Data-Attributes">¶</a></h3><ol>
<li>Time of sample in millisecond</li>
<li>Ankle (shank) acceleration - horizontal forward acceleration (mg)</li>
<li>Ankle (shank) acceleration - vertical (mg)</li>
<li>Ankle (shank) acceleration - horizontal lateral (mg)</li>
<li>Upper leg (thigh) acceleration - horizontal forward acceleration (mg)</li>
<li>Upper leg (thigh) acceleration - vertical (mg)</li>
<li>Upper leg (thigh) acceleration - horizontal lateral (mg)</li>
<li>Trunk acceleration - horizontal forward acceleration (mg)</li>
<li>Trunk acceleration - vertical (mg)</li>
<li>Trunk acceleration - horizontal lateral (mg)</li>
<li>Annotations (see Annotations section)</li>
</ol>
<h4 id="Annotations">Annotations<a class="anchor-link" href="#Annotations">¶</a></h4><p>The meaning of the annotations are as follows:
0: not part of the experiment. For instance the sensors are installed on the user or the user is performing activities unrelated to the experimental protocol, such as debriefing
1: experiment, no freeze (can be any of stand, walk, turn)
2: freeze</p>
<h3 id="Get-Data">Get Data<a class="anchor-link" href="#Get-Data">¶</a></h3>


In [ ]:

daphnet = []
daphnet_thigh = []
daphnet_shank = []
daphnetNames = []
for i in sorted(glob("daphnet/*.txt")):
    daphnetNames.append(i.split("\")[-1])
    daphnet.append(pd.read_csv(i, sep=" ", names=["time", "shank_h_fd", "shank_v", "shank_h_l", "thigh_h_fd", "thigh_v", "thigh_h_l", "trunk_h_fd", "trunk_v", "trunk_h_l", "annotations"]))
    daphnet_thigh.append(daphnet[-1][["time", "thigh_h_fd", "thigh_v", "thigh_h_l", "annotations"]])
    daphnet_shank.append(daphnet[-1][["time", "shank_h_fd", "shank_v", "shank_h_l", "annotations"]])

daphnet[0]




<h4 id="Plots">Plots<a class="anchor-link" href="#Plots">¶</a></h4>


In [ ]:

for i in range(len(daphnet)):
    print(daphnetNames[i])
    fig, axes = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(12, 8))
    fig.suptitle("Data from "+daphnetNames[i])
    plt.xlabel("Time")
    
    ax1 = axes[0][0]
    ax1.plot(daphnet[i]["time"], daphnet[i]["shank_h_fd"], 'r--')
    ax1.set_ylabel("Horizontal Forward Shank Acceleration")
    
    ax2 = axes[0][1]
    ax2.plot(daphnet[i]["time"], daphnet[i]["shank_v"], 'b--')
    ax2.set_ylabel("Vertical Shank Acceleration")
    
    ax3 = axes[1][0]
    ax3.plot(daphnet[i]["time"], daphnet[i]["shank_h_l"], 'g--')
    ax3.set_ylabel("Horizontal Lateral Shank Acceleration")
    
    ax4 = axes[1][1]
    ax4.plot(daphnet[i]["time"], np.sqrt(daphnet[i]["shank_h_l"]**2 + daphnet[i]["shank_v"]**2 + daphnet[i]["shank_h_fd"]**2), 'm--')
    ax4.set_ylabel("Overall Shank Acceleration")
    
    ax1 = axes[2][0]
    ax1.plot(daphnet[i]["time"], daphnet[i]["thigh_h_fd"], 'r--')
    ax1.set_ylabel("Horizontal Forward Thigh Acceleration")
    
    ax2 = axes[2][1]
    ax2.plot(daphnet[i]["time"], daphnet[i]["thigh_v"], 'b--')
    ax2.set_ylabel("Vertical Thigh Acceleration")
    
    ax3 = axes[3][0]
    ax3.plot(daphnet[i]["time"], daphnet[i]["thigh_h_l"], 'g--')
    ax3.set_ylabel("Horizontal Lateral Thigh Acceleration")
    
    ax4 = axes[3][1]
    ax4.plot(daphnet[i]["time"], np.sqrt(daphnet[i]["thigh_h_l"]**2 + daphnet[i]["thigh_v"]**2 + daphnet[i]["thigh_h_fd"]**2), 'm--')
    ax4.set_ylabel("Overall Thigh Acceleration")
    
    plt.show()




<h4 id="Plots-with-Validation-Stuff">Plots with Validation Stuff<a class="anchor-link" href="#Plots-with-Validation-Stuff">¶</a></h4><p>Yes, I know that is unprofessional but sorry, ok.</p>


In [ ]:

for ind in range(len(daphnet)):
    print(daphnetNames[ind])
    df = daphnet[ind]
    sub1 = df[df.annotations == 1]
    sub2 = df[df.annotations == 2]
    
    fig, axes = plt.subplots(6, 2, sharex=True, sharey=True, figsize=(12, 12))
    fig.suptitle("Data from "+daphnetNames[ind])
    plt.xlabel("Time")
    
    labels = ["shank_h_fd", "shank_v", "shank_h_l", "thigh_h_fd", "thigh_v", "thigh_h_l"]
    for i in range(6):
        sig = signal.medfilt(df[labels[i]], 21)

        b, a = signal.butter(5,0.05)
        y = signal.filtfilt(b, a, df[labels[i]])
        
        
        ax1 = axes[i][0]
        ax1.plot(df.time, df[labels[i]])
        ax1.scatter(sub1.time, sub1[labels[i]], c='orange', label="no freeze")
        ax1.scatter(sub2.time, sub2[labels[i]], c='purple', label="freeze")
        ax1.legend()

        ax2 = axes[i][1]
        ax2.plot(df.time, y)
        ax2.scatter(sub1.time, sub1[labels[i]], c='orange', label="no freeze")
        ax2.scatter(sub2.time, sub2[labels[i]], c='purple', label="freeze")
        ax2.legend()
        
    plt.show()




<h3 id="Code-given-by-Dataset">Code given by Dataset<a class="anchor-link" href="#Code-given-by-Dataset">¶</a></h3><p>Translated from Octave in code.py and subject.py (attached)</p>



<h4 id="Subject-Identification-and-Analysis">Subject Identification and Analysis<a class="anchor-link" href="#Subject-Identification-and-Analysis">¶</a></h4><p>using OOP</p>


In [ ]:

from scripts.daphnet import Subject
subjects = Subject.parse(daphnet_thigh, daphnetNames)
subjects




<h4 id="Getting-Data-Results">Getting Data Results<a class="anchor-link" href="#Getting-Data-Results">¶</a></h4>


In [ ]:

hdaphnet = [subject.analyze() for subject in subjects]
hdaphnet



In [ ]:

hyp = []
for i in hdaphnet:
    for j in i: 
        hyp.append(j)

hyp



In [ ]:

idaphnet = [subject.info() for subject in subjects]
idaphnet



In [ ]:

dfs = []
for i in range(17):
    data = np.array(hyp[i])
    data = data[np.where(daphnet_thigh[i].annotations.to_numpy() != 0)]
    y = daphnet_thigh[i][daphnet_thigh[i].annotations != 0]
    y.annotations -= 1
    index = y.time
    df = pd.DataFrame(
        data=np.hstack((data.T, y.annotations)), 
        index=index, 
        columns=["thigh_hf", "thigh_v", "thigh_hl", "y"]
    )
    dfs.append(df)




<h2 id="VGRF-Dataset"><a href="https://physionet.org/content/gaitpdb/1.0.0/">VGRF Dataset</a><a class="anchor-link" href="#VGRF-Dataset">¶</a></h2><h3 id="Data-Description">Data Description<a class="anchor-link" href="#Data-Description">¶</a></h3><p>Parkinson's disease (PD) is one of the most common movement disorders, affecting approximately 1 million Americans (estimates range between 4 and 6.5 million people worldwide) and about 1% of older adults. In the US alone, 60,000 new cases are diagnosed each year. PD is a chronic and progressive neurological disorder that results in tremor, rigidity, slowness, and postural instability. A disturbed gait is a common, debilitating symptom; patients with severe gait disturbances are prone to falls and may lose their functional independence.</p>
<p>This database contains measures of gait from 93 patients with idiopathic PD (mean age: 66.3 years; 63% men), and 73 healthy controls (mean age: 66.3 years; 55% men). The database includes the vertical ground reaction force records of subjects as they walked at their usual, self-selected pace for approximately 2 minutes on level ground. Underneath each foot were 8 sensors (<a href="http://www.infotronic.nl/#CDG">Ultraflex Computer Dyno Graphy, Infotronic Inc.</a>) that measure force (in Newtons) as a function of time. The output of each of these 16 sensors has been digitized and recorded at 100 samples per second, and the records also include two signals that reflect the sum of the 8 sensor outputs for each foot. For details about the format of the data, please see <a href="https://physionet.org/content/gaitpdb/1.0.0/format.txt">this note</a>.</p>
<p>With this information, one can investigate the force record as a function of time and location, derive measures that reflect the center-of-pressure as a function of time, and determine timing measures (e.g., stride time, swing time) for each foot as functions of time. Thus, one can study the stride-to-stride dynamics and the variability of these time series.</p>
<p>This database also includes demographic information, measures of disease severity (i.e., using the Hoehn &amp; Yahr staging and/or the Unified Parkinson's Disease Rating Scale) and other related measures (available in HTML or xls spreadsheet format).</p>
<p>A subset of the database includes measures recorded as subjects performed a second task (serial 7 subtractions) while walking, as in the figure above, which shows excerpts of swing time series from a patient with PD (lower panels) and a control subject (upper panels), under usual walking conditions (at left) and when performing serial 7 subtractions (at right). Under usual walking conditions, variability is larger in the patient with PD (Coefficient of Variation = 2.7%), compared to the control subject (CV = 1.3%). Variability increases during dual tasking in the subject with PD (CV = 6.5%), but not in the control subject (CV = 1.2%). From Yogev et al. (reference [4] below).</p>
<h3 id="Data-format">Data format<a class="anchor-link" href="#Data-format">¶</a></h3><p>Each line contains 19 columns:</p>
<table>
<thead><tr>
<th>Column</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>Time (in seconds)</td>
</tr>
<tr>
<td>2-9</td>
<td><strong>Vertical ground reaction force</strong> (VGRF, in Newton) on each of 8 sensors located under the left foot</td>
</tr>
<tr>
<td>10-17</td>
<td>VGRF on each of the 8 sensors located under the right foot</td>
</tr>
<tr>
<td>18</td>
<td>Total force under the left foot</td>
</tr>
<tr>
<td>19</td>
<td>Total force under the right foot</td>
</tr>
</tbody>
</table>
<p>When a person is comfortably standing with both legs parallel to each
other, sensor locations inside the insole can be described (according
to the <a href="http://www.infotronic.nl/">Infotronic website</a>) as lying
approximately at the following (X,Y) coordinates, assuming that the
origin (0,0) is just between the legs and the person is facing towards
the positive side of the Y axis:</p>
<table>
<thead><tr>
<th>Sensor</th>
<th>X</th>
<th>Y</th>
</tr>
</thead>
<tbody>
<tr>
<td>L1</td>
<td>-500</td>
<td>-800</td>
</tr>
<tr>
<td>L2</td>
<td>-700</td>
<td>-400</td>
</tr>
<tr>
<td>L3</td>
<td>-300</td>
<td>-400</td>
</tr>
<tr>
<td>L4</td>
<td>-700</td>
<td>0</td>
</tr>
<tr>
<td>L5</td>
<td>-300</td>
<td>0</td>
</tr>
<tr>
<td>L6</td>
<td>-700</td>
<td>400</td>
</tr>
<tr>
<td>L7</td>
<td>-300</td>
<td>400</td>
</tr>
<tr>
<td>L8</td>
<td>-500</td>
<td>800</td>
</tr>
<tr>
<td>R1</td>
<td>500</td>
<td>-800</td>
</tr>
<tr>
<td>R2</td>
<td>700</td>
<td>-400</td>
</tr>
<tr>
<td>R3</td>
<td>300</td>
<td>-400</td>
</tr>
<tr>
<td>R4</td>
<td>700</td>
<td>0</td>
</tr>
<tr>
<td>R5</td>
<td>300</td>
<td>0</td>
</tr>
<tr>
<td>R6</td>
<td>700</td>
<td>400</td>
</tr>
<tr>
<td>R7</td>
<td>300</td>
<td>400</td>
</tr>
<tr>
<td>R8</td>
<td>500</td>
<td>800</td>
</tr>
</tbody>
</table>
<p>The X and Y numbers are in an arbitrary coordinate system reflecting
the relative (arbitrarily scaled) positions of the sensors within each
insole. During walking, the sensors inside each insole remain at the
same relative position, but the two feet are no longer parallel to
each other. Thus, this coordinate system enables a calculation of a
proxy for the location of the center of pressure (COP) under each
foot.</p>
<hr/>
<h3 id="Data-file-names">Data file names<a class="anchor-link" href="#Data-file-names">¶</a></h3><p>These follow a common convention, e.g., <code>GaCo01_02.txt</code>  or  <code>JuPt03_06.txt</code>, where</p>
<ul>
<li><p>Ga, Ju or Si – indicate the study from which the data originated:</p>
<ul>
<li>Ga - Galit Yogev et al (<a href="https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1460-9568.2005.04298.x">dual tasking in PD; Eur J Neuro, 2005</a>)</li>
<li>Ju – Hausdorff et al (<a href="https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1460-9568.2007.05810.x">RAS in PD; Eur J Neuro, 2007</a>)</li>
<li>Si - Silvi Frenkel-Toledo et al (<a href="https://onlinelibrary.wiley.com/doi/abs/10.1002/mds.20507">Treadmill walking in PD; Mov Disorders, 2005</a>)</li>
</ul>
</li>
<li><p>Co or Pt: Control subject or a PD Patient respectively</p>
</li>
<li><p>01: Subject number in the group</p>
</li>
<li><p>A walk number of 10  (for the "Ga" study) indicates a dual-task walking,
where the subject was engaged in serial-7 subtraction while walking.</p>
</li>
<li><p>A walk number of 01 refers to a usual, normal walk.</p>
</li>
</ul>
<p>The sampling rate was 100 Hz.</p>
<h3 id="Get-Data">Get Data<a class="anchor-link" href="#Get-Data">¶</a></h3>


In [ ]:

vgrf = []
VGRFnames = []
for i in sorted(glob("vgrf/*.txt")):
    VGRFnames.append(i)
    vgrf.append(pd.read_table(i, names=["time"]+["VGRF_left_"+str(i) for i in range(1, 9)]+["VGRF_right_"+str(i) for i in range(1, 9)]+["VGRF_left", "VGRF_right"]).set_index("time"))

vgrf[0]




<h3 id="Plot-Data">Plot Data<a class="anchor-link" href="#Plot-Data">¶</a></h3>


In [ ]:

dta = vgrf[0]
plt.figure(num=None, figsize=(50, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(dta.index, dta[['VGRF_left']])
plt.figure(num=None, figsize=(50, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(dta.index, dta[['VGRF_right']])
plt.show()

